In [1]:

# coding: utf-8

# In[11]:


#Load dataset from pickled file 

from pickle import load
import numpy as np
import pandas as pd
from keras.models import Sequential, Model, load_model
from keras.layers import TimeDistributed, RepeatVector, LSTM, Dense, Embedding, Input, concatenate
from keras.preprocessing.text import Tokenizer, one_hot, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from gensim.models import Word2Vec
from collections import Counter
from random import randrange
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelBinarizer

considered_size = 5000 #increase this on better machine

stories = load(open('cnn_dataset.pkl', 'rb'))
print('Loaded Stories %d' % len(stories))


articles = []
summaries = []
for story in stories :
    article = story['story']
    new_article = ""
    for item in article :
        new_article += item + " "
    new_article = " ".join(new_article.split())
    articles.append(new_article)
    
    summary = story['highlights']
    new_summary = ""
    for item in summary :
        new_summary += item + " "
    new_summary = " ".join(new_summary.split())
    summaries.append(new_summary)
print(articles[5], summaries[5], sep = "\n")
del stories

article_lengths = []
summary_lengths = []
for article in articles :
    article_lengths.append(len(article.split()))
for summary in summaries :
    summary_lengths.append(len(summary.split()))



a = pd.Series(article_lengths)
b = pd.Series(summary_lengths)
print(a.describe())
print(b.describe())
del a, b, article_lengths, summary_lengths


# In[7]:


all_words = " ".join(articles) + " ".join(summaries)
all_words = text_to_word_sequence(all_words)
vocab_size = len(set(all_words)) #56558, 27178 for only summaries with 60 words

print("Size of vocabulary : ", vocab_size)
del all_words


# In[13]:


print(len(articles), len(summaries))
articles = articles[:considered_size]
summaries = summaries[:considered_size]
print(len(articles), len(summaries))

# tokenized x, y and xy

tok_x = [text_to_word_sequence(xi)[:100] + max(0, 100 - len(text_to_word_sequence(xi)[:100])) * ["__unk__"] for xi in articles]
tok_y = [text_to_word_sequence(yi)[:30] + max(0, 30 - len(text_to_word_sequence(yi)[:30])) * ["__unk__"] for yi in summaries]
tok_xy = [xi + yi for xi, yi in zip(tok_x, tok_y)]

len_tok_x = set([len(xi) for xi in tok_x])
print(len_tok_x)

len_tok_y = set([len(yi) for yi in tok_y])
print(len_tok_y)

print(len(tok_xy[0]))
print(tok_xy[0])

tok_xy.append(5 * ["__unk__"])
print(tok_xy[0])

# word2vec model
model_w2v = Word2Vec(tok_xy, size = 100, min_count = 5)

# size of vocab
words = list(model_w2v.wv.vocab)
vocab_size = len(words)
print("New vocab size",vocab_size)

# replacing unknown words
tok_x = [[word if word in model_w2v.wv.vocab else "__unk__" for word in xi] for xi in tok_x]
tok_y = [[word if word in model_w2v.wv.vocab else "__unk__" for word in yi] for yi in tok_y]

# x values
x_train_article = np.array([[model_w2v[word] for word in xi] for xi in tok_x])
x_train_summary = np.array([[model_w2v[word] for word in yi] for yi in tok_y])

print(x_train_article.shape)
print(x_train_summary.shape)


del tok_x
del tok_xy


# In[ ]:


print(words[:10])

# integer encode
label_encoder = LabelEncoder()
label_encoder.fit(words)

del words

print(label_encoder.transform(["world"]))

# one-hot encode
label_binarizer = LabelBinarizer()
label_binarizer.fit(range(vocab_size))

y_train = np.array([label_binarizer.transform(label_encoder.transform(yi)) for yi in tok_y])
print(y_train.shape)

Using TensorFlow backend.
C:\Users\Varun\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


Loaded Stories 92579
they are the worlds cultural capitals the nerve centers of innovation and the engine rooms of economic growth but could cities also hold the key to cutting carbon emissions longterm a study from the world bank found that the largest cities and urban areas on the planet are now home to roughly million people and spew out some billion tons of greenhouse gasses every year as urban migration continues apace these figures are only expected to rise in the short term while this may initially lead to more pollutants being pumped into the earths atmosphere some experts believe it could work out better in the long term they say that the ecological efficiencies cities can offer aligned with their financial and political influence could lead to the development of more effective ways to curb carbon emissions as the worlds leading environmental figures gather in durban south africa for the united nations conference on climate change cnn asked two urban climate change experts to 

New vocab size 10816


C:\Users\Varun\Anaconda3\lib\site-packages\ipykernel_launcher.py:113: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Users\Varun\Anaconda3\lib\site-packages\ipykernel_launcher.py:114: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


(5000, 100, 100)
(5000, 30, 100)
['driving', 'in', 'pakistans', 'swat', 'valley', 'a', 'region', 'located', 'close', 'to']
[10695]
(5000, 30, 10816)


In [21]:
epochs = 10
batch_size = 32

src_txt_length = 100
sum_txt_length = 30
word_vec_length = 100

# article input model
inputs1 = Input(shape=(src_txt_length, word_vec_length))
# article1 = Embedding(vocab_size, 128)(inputs1)
article2 = LSTM(128)(inputs1)
article3 = RepeatVector(sum_txt_length)(article2)

# summary input model
inputs2 = Input(shape=(sum_txt_length, word_vec_length))
# summ1 = Embedding(vocab_size, 128)(inputs2)

# decoder model
decoder1 = concatenate([article3, inputs2])
decoder2 = LSTM(128)(decoder1)                           # made changes here
outputs = Dense(vocab_size, activation='softmax')(decoder2)      # made changes here

# tie it together [article, summary] [word]
model = Model(inputs=[inputs1, inputs2], outputs=outputs)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['acc'])
model.fit([x_train_article, x_train_summary], y_train, epochs = epochs, batch_size = batch_size)

ValueError: Input arrays should have the same number of samples as target arrays. Found 5000 input samples and 30 target samples.

In [23]:
#vocab_size = ...
epochs = 10
batch_size = 32
src_txt_length = 100
sum_txt_length = 30
word_vec_length = 100
# encoder input model
inputs = Input(shape=(src_txt_length, word_vec_length))
#encoder1 = Embedding(vocab_size, 128)(inputs)
encoder2 = LSTM(128)(inputs)
encoder3 = RepeatVector(sum_txt_length)(encoder2)
# decoder output model
decoder1 = LSTM(128, return_sequences=True)(encoder3)
outputs = TimeDistributed(Dense(vocab_size, activation='softmax'))(decoder1)
# tie it together
model = Model(inputs=inputs, outputs=outputs)
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.fit(x_train_article, y_train, epochs = epochs, batch_size = batch_size)

Epoch 1/10
5000/5000 [==============================] - 358s 72ms/step - loss: 7.3498
Epoch 2/10
5000/5000 [==============================] - 350s 70ms/step - loss: 7.0523
Epoch 3/10
5000/5000 [==============================] - 353s 71ms/step - loss: 7.0507
Epoch 4/10
5000/5000 [==============================] - 357s 71ms/step - loss: 7.0504
Epoch 5/10
5000/5000 [==============================] - 378s 76ms/step - loss: 7.0500
Epoch 6/10
5000/5000 [==============================] - 360s 72ms/step - loss: 7.0512
Epoch 7/10
5000/5000 [==============================] - 361s 72ms/step - loss: 7.0507
Epoch 8/10
5000/5000 [==============================] - 355s 71ms/step - loss: 7.0508
Epoch 9/10
5000/5000 [==============================] - 363s 73ms/step - loss: 7.0504
Epoch 10/10
5000/5000 [==============================] - 352s 70ms/step - loss: 7.0503


In [25]:
a = model.predict(np.array([x_train_article[0]]))
print(a)

[[[  2.65412312e-02   8.76063388e-03   8.91163218e-05 ...,   8.81302185e-05
     1.85262998e-05   4.66172387e-05]
  [  7.77303204e-02   1.91469472e-02   5.80466731e-05 ...,   5.77768515e-05
     8.02972227e-06   2.58804430e-05]
  [  8.95150155e-02   2.10824180e-02   5.30885409e-05 ...,   5.28941018e-05
     6.89927629e-06   2.30971345e-05]
  ..., 
  [  9.14949030e-02   2.13951990e-02   5.23056951e-05 ...,   5.21224028e-05
     6.73022896e-06   2.26679131e-05]
  [  9.14949030e-02   2.13951990e-02   5.23056951e-05 ...,   5.21224028e-05
     6.73022896e-06   2.26679131e-05]
  [  9.14949030e-02   2.13951990e-02   5.23056951e-05 ...,   5.21224028e-05
     6.73022896e-06   2.26679131e-05]]]


In [26]:
#k = model.predict(np.array([inp]))
inverted = label_encoder.inverse_transform([np.argmax(i) for i in a[0]])
print(inverted)

['__unk__' '__unk__' '__unk__' '__unk__' '__unk__' '__unk__' '__unk__'
 '__unk__' '__unk__' '__unk__' '__unk__' '__unk__' '__unk__' '__unk__'
 '__unk__' '__unk__' '__unk__' '__unk__' '__unk__' '__unk__' '__unk__'
 '__unk__' '__unk__' '__unk__' '__unk__' '__unk__' '__unk__' '__unk__'
 '__unk__' '__unk__']
